In [1]:
import json
import random as rd
from unidecode import unidecode
import nltk as nltk
from nltk.tag.perceptron import PerceptronTagger
from nltk.corpus import floresta
from nltk.corpus import stopwords
import re

In [2]:
file = 'anotados_3929.json'
with open('C:\\Users\\92007848\\Documents\\AIProj\\AiTeste\\NLTKAlpha\\Resources\\'+file,encoding='Utf-8') as json_file:
    conteudo = json.load(json_file)

rd.shuffle(conteudo)
linha = 0
anotacao = 0
##Analise do dataset
print(conteudo[linha])
print('')
print(conteudo[linha]['content'])
print('')
print(conteudo[linha]['intent'])
print('')
print('Anotações')
print(conteudo[linha]['annotation'])
print('')


In [ ]:
##Criar um ID para as entradas do dataSet
conteudo_id = []
for line , data in enumerate(conteudo):
    element = {line: data}
    conteudo_id.append(element)

linha = 21
anotacao = 0
id = 0
##Analise do dataset
print(conteudo_id[linha])
print('')
print(conteudo_id[linha][id]['content'])
print('')
print(conteudo_id[linha][id]['intent'])
print('')
print('Anotações')
print(conteudo_id[linha][id]['annotation'][anotacao])
print('')

In [ ]:
#Separando os conteudos de cada trecho
#Separando o content, annotation e as keywords
contents = {}
annotations = {}
keywords = []
for number , data in enumerate(conteudo_id):
    contents[number] = data[number]['content']
    annotations[number] = data[number]['annotation']   
    for annotation in data[number]['annotation']:
        #print(unidecode(annotation[0]['points'][0]['text'].lower().replace('\'','').replace('\"','')) if annotation[0]['label'][0] != 'CUMPRIMENTO' else 'cumprimento')
        keywords.append(re.sub(r'[."0-9]','',unidecode(annotation[0]['points'][0]['text'].lower().replace('\'','').replace('\"',''))) if annotation[0]['label'][0] != 'CUMPRIMENTO' else 'cumprimento')
    



In [ ]:
print(keywords)

In [ ]:
#Baixando o dataSet pra treino
#nltk.corpus.mac_morpho.words()
#nltk.corpus.mac_morpho.tagged_words()
tagged = nltk.corpus.mac_morpho.tagged_words()

In [ ]:
CorpusList = []
for new in tagged:
    if new[0] != '':
        CorpusList.append((unidecode(new[0]),new[1]))

In [ ]:
keywords_tuple = []
stop_words = stopwords.words('portuguese')
for keyword in keywords:
    if keyword != 'cumprimento':
        if len(keyword.split()) > 1:
            keyword_split = [y for y in keyword.split()]
            for key in keyword_split:
                if key not in stop_words:
                    keywords_tuple.append((key,'K'))
                else:
                    keywords_tuple.append((key,'STOP'))
        else:        
            keywords_tuple.append((keyword,'K'))

In [ ]:
keywords_tuple

In [ ]:
#Tirando do dataSet mac_morpho as palavras que existem no meu dataSet custom
import time
start = time.time()
print(len(CorpusList))
for i, key in enumerate(keywords_tuple):
    for j, data in enumerate(CorpusList):
        if data[0] == key[0]:
            CorpusList.pop(j)
    print(i+1,' de ',len(keywords_tuple))        
print(len(CorpusList))
print(time.time()-start)

In [ ]:
for key in keywords_tuple:
    CorpusList.append(key)

In [ ]:
lista = ['-']
for x in CorpusList:
    if x[0] in lista:
        print(x)

for x in keywords_tuple:
    if x[0] in lista:
        print(x)

In [ ]:
rd.shuffle(CorpusList)

In [ ]:
CorpusList.remove(('-','K'))

In [ ]:
taglist = []
taglist.append(CorpusList)

In [ ]:
CorpusList

In [ ]:
tagger = PerceptronTagger(load=False) # don't load existing model
tagger.train(taglist)

def process(sent):
    sent = nltk.word_tokenize(unidecode(sent.lower()))
    stop_words = stopwords.words('portuguese')
    sent = tagger.tag(sent)
    new_sent = []
    returner = []
    aux_sent = ''
    pattern = '''NP:  {<K><STOP>*<K>+} 
                      {<K>+}'''
    cp = nltk.RegexpParser(pattern)
    cs = cp.parse(sent)
    for i in cs:
        if len(i) < 2 and i[0][1] == 'K':
            returner.append(i[0][0])
        elif len(i) > 2:
            for j in i:
                aux_sent += j[0]+' '
            returner.append(aux_sent)
            aux_sent = ''
        elif isinstance(i[0],tuple):
            returner.append(i[0][0]+' '+i[1][0])
    return returner        
    

def process(sent):
    sent = nltk.word_tokenize(unidecode(sent.lower()))
    stop_words = stopwords.words('portuguese')
    sent = tagger.tag(sent)
    return sent 

text = "Bom dia. Na página 5 da aula 04, tem o fluxograma sobre a estrutura de repetição while, conforme anexo. Ao chegar na parte da condição (cont>10), o algoritmo já irá finalizar a lógica pois, iniciando o contador com zero (cont<-0) este sempre será menor que 10, finalizando o algoritmo. É esse o raciocínio? Atc."
sent = process(text)
print(sent)

In [ ]:
#Salva um modelo
from sklearn.externals import joblib
joblib.dump(tagger, 'C:\\Users\\92007848\\Documents\\AIProj\\AiTeste\\NLTKAlpha\\Resources\\Models\\AlphaTagger3_003.pkl')

In [ ]:
pattern = '''NP:  {<K><STOP>*<K>+} 
                  {<K>+}'''

In [ ]:
cp = nltk.RegexpParser(pattern)
cs = cp.parse(sent)


In [ ]:
for i in cs:
    print(type(i[0]))

In [ ]:
for i in cs:
    if len(i) < 2:
        print(i)
        print(i[0][0])

In [ ]:
for i in cs:
    if (len(i) > 2 or (len(i) < 2 and i[0][1] == 'K')):
        print(i[0][1])
        for j in i:
            print(j)
        print('')   

In [ ]:
class NLTKProcessor:
    def __init__(self, stop_words = None, pattern: None, tokenizer = lambda x: x.split(), ):
        self.tagger = PerceptronTagger()
        self.stop_words = stop_words
        self.pattern = pattern
        self.tokenizer = tokenizer
        
    def process(sent):
        sent = self.tokenizer(unidecode(sent.lower()))
        sent = self.tagger.tag(sent)
        returner = []
        aux_sent = ''
        cp = nltk.RegexpParser(self.pattern)
        cs = cp.parse(sent)
        for i in cs:
            if len(i) < 2 and i[0][1] == 'K':
                returner.append(i[0][0])
            elif len(i) > 2:
                for j in i:
                    aux_sent += j[0]+' '
                returner.append(aux_sent)
                aux_sent = ''
            elif isinstance(i[0],tuple):
                returner.append(i[0][0]+' '+i[1][0])
        return returner  